# Calculate sidewalk width
Using centerline extraction (also called skeleton line, axis line, or medial line extraction)

In [ ]:
# Add project src to path.
import set_path

import os

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pathlib

from shapely.ops import linemerge
from shapely.geometry import box

import geopandas as gpd
from centerline.geometry import Centerline

from tqdm.notebook import tqdm_notebook
tqdm_notebook.pandas()

import upcp.utils.bgt_utils as bgt_utils
import upcp.utils.las_utils as las_utils

import upc_sw.poly_utils as poly_utils

In [ ]:
# Paths
pc_data_folder = '../datasets/pointclouds/'
out_folder = '../datasets/output/'

# A CRS tells Python how those coordinates relate to places on the Earth. Rijksdriehoek = epsg:28992
crs = 'epsg:28992' #local crs

# Shapely tolerance argument
simplify_tolerance = 0.2

# Min area size of a sidewalk polygon in meters
min_area_size = 5

# Max linestring length in meters
max_line_length = 5

## Read the sidewalk and obstacle data

In [ ]:
# Read sidewalk with obstacle data
df = gpd.read_file('../datasets/obstacles/sidewalks_with_obstacles.shp')

## Merge the sidewalk polygons

In [ ]:
df = gpd.GeoDataFrame(geometry=gpd.GeoSeries([geom for geom in df.unary_union.geoms]))
df = gpd.GeoDataFrame(df.geometry.explode(index_parts=True))

In [ ]:
# Ignore sidewalk polygons that are too small
df = df[df.geometry.area > min_area_size]

## Calculate the centerlines

In [ ]:
def get_centerlines(row_geometry):
    ''' Save a NaN value when centerline calculation fails. '''
    try:
        x = Centerline(row_geometry)
    except Exception as e:
        print(e) # TODO also print rows.name[0] ??
        x = np.nan
    return x

In [ ]:
# if you get an error here, make sure you use tqdm>=4.61.2
df['centerlines'] = df.progress_apply(lambda row: get_centerlines(row.geometry), axis=1)
df = df.set_geometry('centerlines')

## Remove short line ends and dead-ends

In [ ]:
df['centerlines'] = df['centerlines'].progress_apply(linemerge)

In [ ]:
df['centerlines'] = df['centerlines'].progress_apply(lambda x: poly_utils.remove_short_lines(x, max_line_length))

## Get sidewalk widths

In [ ]:
df['centerlines'] = df['centerlines'].progress_apply(lambda row: row.simplify(simplify_tolerance, preserve_topology=True))

In [ ]:
df['segments'] = df['centerlines'].progress_apply(poly_utils.get_segments)

In [ ]:
df[['avg_distances', 'min_distances']] = df.progress_apply(lambda row: poly_utils.get_avg_distances(row), axis=1)

In [ ]:
data = {'geometry': [], 'avg_width': [], 'min_width': []}

for i, row in df.iterrows():
    
    for segment in row.segments:
        data['geometry'].append(segment)
    
    for avg_distance in row.avg_distances:
        data['avg_width'].append(avg_distance * 2)
    
    for min_distance in row.min_distances:
        data['min_width'].append(min_distance * 2)
        
df = pd.DataFrame(data)
df = gpd.GeoDataFrame(df, crs=crs, geometry='geometry')

In [ ]:
# Width in meters
df['avg_width'] = np.around(df['avg_width'], decimals=1)
df['min_width'] = np.around(df['min_width'], decimals=1)

## Check coverage of point cloud data on sidewalks

In [ ]:
all_tiles = las_utils.get_tilecodes_from_folder(f'{pc_data_folder}m3c2/')

In [ ]:
tile_dimensions = [poly_utils.tilecode_to_poly(tile) for tile in all_tiles]
df_tile_dimensions = gpd.GeoDataFrame(crs=crs, geometry=tile_dimensions)

# When True, take the union of the polygons
if len(tile_dimensions) > 1:
    df_tile_dimensions = gpd.GeoDataFrame(geometry=gpd.GeoSeries([geom for geom in df_tile_dimensions.unary_union.geoms]))
    df_tile_dimensions = gpd.GeoDataFrame(df_tile_dimensions.geometry.explode(index_parts=True))

In [ ]:
# Check if line segment intersects with point cloud tile
df['pc_coverage'] = df.apply(lambda row: df_tile_dimensions.intersects(row.geometry).any(), axis=1)

## Store output

In [ ]:
df = df.to_crs('EPSG:4326')

pathlib.Path(out_folder).mkdir(parents=True, exist_ok=True)
with open(f'{output_folder}intermediate_output_segments.geojson', 'w') as f:
    f.write(df.to_json())